<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/data_connectors/DatabaseReaderDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Database Reader

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
%pip install llama-index-readers-database

In [ ]:
!pip install llama-index

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
from __future__ import absolute_import

# My OpenAI Key
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("open ai api key: ")

from llama_index.readers.database import DatabaseReader
from llama_index.core import VectorStoreIndex

In [ ]:
!docker run --name my_postgres -e POSTGRES_USER=postgres -e POSTGRES_PASSWORD=FakeExamplePassword -e POSTGRES_DB=postgres -p 5432:5432 -d postgres


In [ ]:
# Initialize DatabaseReader object with the following parameters:

db = DatabaseReader(
    scheme="postgresql",  # Database Scheme
    host="localhost",  # Database Host
    port="5432",  # Database Port
    user="postgres",  # Database User
    password="FakeExamplePassword",  # Database Password
    dbname="postgres",  # Database Name
)

In [ ]:
### DatabaseReader class ###
# db is an instance of DatabaseReader:
print(type(db))
# DatabaseReader available method:
print(type(db.load_data))

### SQLDatabase class ###
# db.sql is an instance of SQLDatabase:
print(type(db.sql_database))
# SQLDatabase available methods:
print(type(db.sql_database.from_uri))
print(type(db.sql_database.get_single_table_info))
print(type(db.sql_database.get_table_columns))
print(type(db.sql_database.get_usable_table_names))
print(type(db.sql_database.insert_into_table))
print(type(db.sql_database.run_sql))
# SQLDatabase available properties:
print(type(db.sql_database.dialect))
print(type(db.sql_database.engine))

In [ ]:
### Testing DatabaseReader
### from SQLDatabase, SQLAlchemy engine and Database URI:

# From SQLDatabase instance:
print(type(db.sql_database))
db_from_sql_database = DatabaseReader(sql_database=db.sql_database)
print(type(db_from_sql_database))

# From SQLAlchemy engine:
print(type(db.sql_database.engine))
db_from_engine = DatabaseReader(engine=db.sql_database.engine)
print(type(db_from_engine))

# From Database URI:
print(type(db.uri))
db_from_uri = DatabaseReader(uri=db.uri)
print(type(db_from_uri))

In [ ]:
from sqlalchemy import Table, Column, Integer, String, MetaData

metadata = MetaData()
users_table = Table(
    "users", metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String, nullable=False),
    Column("age", Integer, nullable=False)
)

# Create the table
metadata.create_all(engine)
print("Users table created.")


In [ ]:
with engine.connect() as conn:
    result = conn.execute(text(
        "SELECT * FROM information_schema.tables WHERE table_schema = 'public';"
    ))
    print("All tables in the public schema:")
    for row in result:
        print(row)


In [ ]:
with engine.connect() as conn:
    # Start a transaction
    trans = conn.begin()
    try:
        conn.execute(users_table.insert(), [
            {"name": "Alice", "age": 25},
            {"name": "Bob", "age": 30},
            {"name": "Charlie", "age": 17},
            {"name": "Diana", "age": 22}
        ])
        # Commit the transaction
        trans.commit()
        print("Sample data inserted into the users table.")
    except Exception as e:
        trans.rollback()  # Rollback in case of an error
        print(f"Error: {e}")


In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM users;"))
    print("Number of rows in users table:", result.scalar())


In [ ]:
from sqlalchemy import text

# Execute the query using sqlalchemy.text
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM users;"))
    for row in result:
        print(row)


In [ ]:
# The below SQL Query example returns a list values of each row
# with concatenated text from the name and age columns
# from the users table where the age is greater than or equal to 18

query = f"""
    SELECT
        CONCAT(name, ' is ', age, ' years old.') AS text
    FROM public.users
    WHERE age >= 18
    """

In [ ]:
# Please refer to llama_index.utilities.sql_wrapper
# SQLDatabase.run_sql method
texts = db.sql_database.run_sql(command=query)

# Display type(texts) and texts
# type(texts) must return <class 'list'>
print(type(texts))

# Documents must return a list of Tuple objects
print(texts)

In [ ]:
texts

In [ ]:
# Please refer to llama_index.readers.database.DatabaseReader.load_data
# DatabaseReader.load_data method
documents = db.load_data(query=query)

# Display type(documents) and documents
# type(documents) must return <class 'list'>
print(type(documents))

# Documents must return a list of Document objects
print(documents)

In [ ]:
index = VectorStoreIndex.from_documents(documents)

In [ ]:
index